In [1]:
import re
import pickle
import json
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import math
from difflib import SequenceMatcher
import pandas as pd

In [2]:
ref_dict = pickle.load(open("paper_id_references_dict.pickle", "rb"))

In [4]:
paper_id_arnet_id = pickle.load(open("pickles_data/paper_id_arnet_id.pickle","rb"))

In [8]:
acl_title = pickle.load(open("pickles_data/acl_titles_year_auth.pkl","rb"))

In [6]:
paper_refs = {}
print(len(list(paper_id_arnet_id.keys())))

for key in paper_id_arnet_id :
    try :
        paper_refs[key] = ref_dict[paper_id_arnet_id[key]]
    except :
        pass

print(len(list(paper_refs.keys())))

1163
967


In [9]:
author_level = pickle.load(open("pickles/author_level.pickle", "rb"))

In [10]:
list(author_level.keys())[0]

'P14-1066'

In [11]:
mapped_arnet_ids_to_acl = []
id_to_name = {}
name_to_id = {}
for paper , citations in author_level.items():
    for citation in citations:
        arnet_id = citation['arnet_id']
        if arnet_id not in id_to_name :
            id_to_name[arnet_id] = []
        if(citation['paper_name'] not in id_to_name[arnet_id]):
            id_to_name[arnet_id].append(citation['paper_name'])
        name_to_id[citation['paper_name']] = arnet_id
        mapped_arnet_ids_to_acl.append(arnet_id)

In [12]:
for key in paper_id_arnet_id :
    arnet_id = paper_id_arnet_id[key]
    if(arnet_id in paper_refs) :
        id_to_name[arnet_id] = acl_title[key]['paper_name']
        name_to_id[acl_title[key]['paper_name']] = arnet_id

In [16]:
mapped_arnet_ids = list(set(mapped_arnet_ids_to_acl).intersection(set(ref_dict.keys())))

In [17]:
count = 0 
non_uniq_arnet_ids = []
for key in mapped_arnet_ids_to_acl :
    if(key in mapped_arnet_ids) :
        non_uniq_arnet_ids.append(key)
        count+=1
print(count)

25083


In [18]:
len(list(set(mapped_arnet_ids_to_acl)))

13056

In [19]:
len(mapped_arnet_ids)

10564

In [21]:
refs = {}
for id in mapped_arnet_ids :
    refs[id] = ref_dict[id]
    
for key in paper_refs :
    refs[key] = paper_refs[key]
    
print(len(list(refs.keys())))

11531


In [22]:
pickle.dump(refs, open("pickles_data/graph.pkl", "wb"))

In [23]:
pickle.dump(id_to_name, open("pickles_data/arnetids_to_papername.pkl", "wb"))

In [24]:
pickle.dump(name_to_id, open("pickles_data/papername_to_arnetids.pkl", "wb"))

In [22]:
f = open("pickles_data/total_titles.txt")

In [23]:
data = []
for line in f :
    data.append(line)

In [25]:
titles = pickle.load(open("pickles_data/citation_titles.pkl", "rb"))

In [26]:
len(titles)

1181

In [29]:
def get_citations(folder):
    citation_list = {}
    for file in os.listdir(folder) :
        tree = ET.parse(folder+file)
        root = tree.getroot()
        id = file[:8]
        for element in root.iterfind("algorithm"):
            if(element.attrib['name']=="ParsCit"):
                citlist = element.getchildren()
                cits = citlist[0].getchildren()
                citations = []
                for cit in cits:
                    cit_dict = {}
                    if(cit.attrib['valid']=="true"):
                        try :
                            title = cit.find('title').text.lower()
                        except :
                            title = cit.find('rawString').text.lower()
                        cit_dict['title'] = title
                        cit_dict['cit'] = cit
                        citations.append(cit_dict)
                
                citation_list[id] = citations
            
    return citation_list 

In [30]:
citations = get_citations("xmls/")

In [31]:
titles = []
for id in citations :
    papers = citations[id]
    for paper in papers :
        titles.append(paper['title'])

In [32]:
len(titles)

31943

In [34]:
len(non_uniq_arnet_ids)

25083

In [35]:
titles_arnet = []
for id in non_uniq_arnet_ids :
    titles_arnet.append(id_to_name[id])

In [38]:
tags = pickle.load(open("pickles_data/baseline_tags.pkl", "rb"))

In [39]:
baselines = []
for id in tags :
    for paper in tags[id] :
        if(paper['tag']==1) :
            baselines.append(paper['paper_name'])

In [40]:
print(len(baselines))

2953


In [41]:
count = 0
for title in baselines :
    if title in titles_arnet :
        count+=1

print(count)

2101


In [42]:
print(len(list(set(titles))))

16944


In [43]:
titles_arnet_v2 = []
for id in mapped_arnet_ids_to_acl :
    titles_arnet_v2.append(id_to_name[id])

In [44]:
len(titles_arnet_v2)

29973

In [46]:
count = 0
list_base = []
for title in baselines :
    if(title in titles_arnet_v2) :
        list_base.append(title)
        count+=1
print(count)

2424


In [47]:
print(len(set(list_base)))

1774


In [48]:
print(len(set(baselines)))

2178
